# GridLAB-D GeoData Subcommand

As of version 4.2.21, HiPAS GridLAB-D supports the handling of geographic data.  This command checks the version of GridLAB-D.

In [1]:
!gridlabd --version

HiPAS GridLAB-D 4.2.21-210512


The `geodata` subcommand uses the general syntax `gridlabd geodata OPTIONS DIRECTIVE [ARGUMENTS]`.  

## Creating location data

There are two basic types of geodata entities in GridLAB-D: 

1. an unordered collection of points each specified by a `latitude,longitude` tuple; and 

1. an ordered series of waypoints along a path specified by a sequence of `latitude,longitude` tuples.  

The interpretation of a the entity is left to the dataset processor, but it can often been specified using the `location` or `position` keys, respectively.  A geodata entity can be converted from one to another by simply changing the key (see Geodata Indexing below).

The `create` directive is used to create a new geodata entity.  The general syntax is `gridlabd geodata create LOCATIONS ...`.

There are two methods of introducing locations.  The first method introduces one or more `latitude,longitude` tuples directly in the command line.  For example, the following command creates a dataset with the approximate location of SLAC's main gate.

In [2]:
!gridlabd geodata create 37.415,-122.201

id,latitude,longitude
0,37.415,-122.201


Multiple locations can be introduced by adding them to the command, for example:

In [3]:
!gridlabd geodata create 37.410,-122.201 37.420,-122.201

id,latitude,longitude
0,37.41,-122.201
1,37.42,-122.201


The second method uses an input file with locations and associated data, such as this example CSV file:

In [4]:
!head -n 4 path_example.csv

latitude,longitude,configuration,pole_height
37.415045141688054,-122.2056472090359,flat3,15.0
37.414698020593065,-122.20848749028133,sideT,15.0
37.414454093051745,-122.21044282065421,sideT,15.0


To use this file, the following command can be used:

In [5]:
!gridlabd geodata create path_example.csv | head -n 4

id,latitude,longitude,configuration,pole_height
0,37.41505,-122.20565,flat3,15.0
1,37.4147,-122.20849,sideT,15.0
2,37.41445,-122.21044,sideT,15.0


Note that if multiple locations are provided they are sequences in the order in which they are presented, including if locations are provided directly on the command line or from data files.

If no location information is given, then the geodata is read from `/dev/stdin`, e.g.,

In [6]:
!head -n 4 path_example.csv | cut -f1-2 -d, | gridlabd geodata create

id,latitude,longitude
0,37.41505,-122.20565
1,37.4147,-122.20849
2,37.41445,-122.21044


## Some useful options

The default precision with which latitudes and longitudes are output is 5 decimals, which is approximately 1 meter resolution. You can change the precision with which latitudes and longitudes are output using the `-p` or `--precision` option, e.g.,

In [7]:
!gridlabd geodata create 37.410,-122.201 37.420,-122.201 -p 4

id,latitude,longitude
0,37.41,-122.201
1,37.42,-122.201


The default field separator for RAW output is a space. You can change this to any string using the `--fieldsep=STRING` option, e.g.,

In [8]:
!gridlabd geodata create 37.410,-122.201 37.420,-122.201 -f RAW --fieldsep=:

37.41:-122.201
37.42:-122.201


Similarly the default record separator for RAW output is a newline. You can change this to any string using the `--recordsep=STRING` option, e.g.,

In [9]:
!gridlabd geodata create 37.410,-122.201 37.420,-122.201 -f RAW --recordsep=' ; '

37.41 -122.201 ; 37.42 -122.201


## Configurations
There are three locations where configuration settings are maintain: (1) system, (2) user, and (3) local.  They are consulted in this order so that the system configuration overrides the default configuration, the user configuration override the system, and the local configuration overrides the user configuration.  

By default the configuration files are named `geodata.conf`.  The system configuration is stored in `$GLD_ETC/geodata/geodata.conf` folder.  The user configuration is stored in `$HOME/.gridlabd/geodata/geodata.conf` and the local configuration is stored in `$PWD/geodata.conf`. 

The default configuration file name can be changed using the `-C FILENAME` or `--configfile FILENAME` option.

You can manage the current configuration using the `config` directive, e.g., to set the local configuration parameter `name` to `value`, use the `set` option

In [10]:
!gridlabd geodata config set name local_value

WARNING [geodata/None]: config file '/Users/dchassin/Documents/GitHub/slacgismo/gridlabd/gldcore/geodata/docs/geodata.conf' is not found


Note that if the file in which the parameter is stored does not already exist, you will get a warning before it is created.

To get the value, use the `get` option:

In [11]:
!gridlabd geodata config get name

local_value


To show all the configuration values, use the `show` option:

In [12]:
!gridlabd geodata config show

name='local_value'


To set a user configuration, use the `user.` prefix, e.g.,

In [13]:
!gridlabd geodata config set user.name user_value
!gridlabd geodata config show

name='local_value'
user.name='user_value'


The same syntax is used for system configuration values, e.g.,

In [14]:
!gridlabd geodata config set system.name system_value
!gridlabd geodata config show

name='local_value'
user.name='user_value'
system.name='system_value'


To remove a value, use the `unset` option, e.g.,

In [15]:
!gridlabd geodata config unset name
!gridlabd geodata config show

user.name='user_value'
system.name='system_value'


## Geodata formatting

The output format can be changed using the `-f` or `--format` option. The valid formats are `CSV`, `JSON`, `RAW`, and `TABLE`.  `CSV` format is the default, but can be specified as follows:

In [16]:
!gridlabd geodata create 37.410,-122.201 37.420,-122.201 -f CSV

id,latitude,longitude
0,37.41,-122.201
1,37.42,-122.201


JSON output looks like this:

In [17]:
!gridlabd geodata create 37.410,-122.201 37.420,-122.201 -f JSON

[{"latitude":37.41,"longitude":-122.201},{"latitude":37.42,"longitude":-122.201}]


RAW output is generated as follows

In [18]:
!gridlabd geodata create 37.410,-122.201 37.420,-122.201 -f RAW

37.41 -122.201
37.42 -122.201


TABLE output is generated for easy reading:

In [19]:
!gridlabd geodata create 37.410,-122.201 37.420,-122.201 -f TABLE

    latitude  longitude
id                     
0      37.41   -122.201
1      37.42   -122.201


Output formats may include an ordered field list, such as

In [20]:
!gridlabd geodata create 37.410,-122.201 37.420,-122.201 -f CSV:longitude,latitude

id,longitude,latitude
0,-122.201,37.41
1,-122.201,37.42


## Geodata indexing

The output can be indexed using one of several standards keys, or keys from the data. The default key is `id` which is the row number, as seen above. This key can be explicitly specified as follows:

In [21]:
!gridlabd geodata create 37.410,-122.201 37.420,-122.201 -k id

id,latitude,longitude
0,37.41,-122.201
1,37.42,-122.201


The `location` key generates a geohash code:

In [22]:
!gridlabd geodata create 37.410,-122.201 37.420,-122.201 -k location

location,latitude,longitude,id
9q9hg4rcn36n,37.41,-122.201,0
9q9hghpgq3d4,37.42,-122.201,1


The `position` key generates a distance index, treating the data rows as a series of waypoints along a path. When this key is used, the distance and heading also generated.

In [23]:
!gridlabd geodata create 37.410,-122.201 37.420,-122.201 -k position

position,latitude,longitude,id,distance,heading
0,37.41,-122.201,0,0.0,
1111,37.42,-122.201,1,1111.95,0.0


Any field or set of fields may be used for indexing, e.g.,

In [24]:
!gridlabd geodata create 37.410,-122.201 37.420,-122.201 -k latitude,longitude

latitude,longitude,id
37.41,-122.201,0
37.42,-122.201,1


## Path waypoints

Waypoints can be generated along a path using the `-r` or `--resolution` option.  For example, 250-meter waypoints are generated using the following syntax:

In [45]:
!gridlabd geodata create 37.410,-122.201 37.420,-122.201 -r 250 -f TABLE

          latitude  longitude   id  distance  heading
position                                             
0         37.41000   -122.201  0.0      0.00      NaN
250       37.41225   -122.201  NaN    250.00      0.0
500       37.41450   -122.201  NaN    500.00      0.0
750       37.41674   -122.201  NaN    750.00      0.0
1000      37.41899   -122.201  NaN   1000.00      0.0
1111      37.42000   -122.201  1.0   1111.95      0.0


The `distance` and `heading` columns are added to provide the distance and heading to the waypoint from the last location. When using waypoints, the key is automatically set to `position`, which provides the distance from the first point.  The use of other keys is not supported. In addition, any waypoint added in the process is not assigned a row `id` in order to protect the original row ids. 

The index can be changed with the `-k` or `--key` options using a pipe, e.g.,

In [41]:
!gridlabd geodata create 37.410,-122.201 37.420,-122.201 -r 250 | gridlabd geodata create -k location -f TABLE

              position  latitude  longitude   id  distance  heading
location                                                           
9q9hg4rcn36n         0  37.41000   -122.201  0.0      0.00      NaN
9q9hg4xynkfh       250  37.41225   -122.201  NaN    250.00      0.0
9q9hg5pgq261       500  37.41450   -122.201  NaN    500.00      0.0
9q9hg5xbq3fh       750  37.41674   -122.201  NaN    750.00      0.0
9q9hg5zvqm61      1000  37.41899   -122.201  NaN   1000.00      0.0
9q9hghpgq3d4      1111  37.42000   -122.201  1.0   1111.95      0.0


## Output

By default all output is written to `/dev/stdout`.  The output can be written to a file using the `-o` or `--output` option, e.g.,

In [27]:
!gridlabd geodata create 37.410,-122.201 37.420,-122.201 -o /tmp/test.csv
!cat /tmp/test.csv

id,latitude,longitude
0,37.41,-122.201
1,37.42,-122.201


----
# Datasets

The `merge` directive is used to merge a dataset into an existing geodata entity.  The general syntax is `gridlabd geodata merge -D DATASET OPTIONS`

## Distance

The `distance` dataset provide great-circle distance calculations.  For example, the following calculated the distance from the first point in the series.

In [28]:
!gridlabd geodata merge -D distance 37.410,-122.201 37.420,-122.201

id,latitude,longitude,distance
0,37.41,-122.201,0.0
1,37.42,-122.201,593.0


The `distance` dataset support different units, include `meters` or `m`, `kilometers` or `km`, `feet` or `ft`, `yards` or `yd`, `miles` or `mi`.  For example, the following calculates the distances in feet:

In [29]:
!gridlabd geodata merge -D distance 37.410,-122.201 37.420,-122.201 --units=feet

id,latitude,longitude,distance
0,37.41,-122.201,0.0
1,37.42,-122.201,1944.0


Note that the default precision with which distances are calculated in 0 decimals. Thus the distance in `km` is rounded up to `1.0`:

In [30]:
!gridlabd geodata merge -D distance 37.410,-122.201 37.420,-122.201 --units=km

id,latitude,longitude,distance
0,37.41,-122.201,0.0
1,37.42,-122.201,1.0


The precision can be changed thus: 

In [31]:
!gridlabd geodata merge -D distance 37.410,-122.201 37.420,-122.201 --units=km --precision=2

id,latitude,longitude,distance
0,37.41,-122.201,0.0
1,37.42,-122.201,0.59


## Address

You can use the `address` dataset to perform address resolution operations, both to and from latitude and longitude.

To obtain the address at a location, use the following command

In [32]:
!gridlabd geodata merge -D address 37.415,-122.201

id,latitude,longitude,address
0,37.415,-122.201,"Stanford Linear Accelerator Center National Accelerator Laboratory, Sand Hill Road, Menlo Park, San Mateo County, California, 94028, United States"


To find the latitude and longitude of an address, use the command:

In [33]:
!gridlabd geodata merge -D address --reverse "2575 Sand Hill Road, Menlo Park CA"

id,index,address,longitude,latitude
0,0,"2575 Sand Hill Road, Menlo Park CA",-122.20118,37.41546


You can perform both operations to resolve the "official" address from an incomplete address:

In [34]:
!gridlabd geodata merge -D address --reverse "2575 Sand Hill Road, Menlo Park CA" | gridlabd geodata merge -D address

id,index,address,longitude,latitude
0,0,"Stanford Linear Accelerator Center National Accelerator Laboratory, Sand Hill Road, Menlo Park, San Mateo County, California, 94028, United States",-122.20118,37.41546


## Elevation
Elevation data can be obtained using the `elevation` data set, e.g.,  

In [66]:
!gridlabd geodata merge -D elevation 37.415,-122.201 

id,latitude,longitude,elevation
0,37.415,-122.201,83.0


Elevations can be calculated in other units using the `--units=UNIT` options, e.g.,

In [67]:
!gridlabd geodata merge -D elevation 37.415,-122.201 --units=feet

id,latitude,longitude,elevation
0,37.415,-122.201,272.0


Valid units are `m` or `meters`, and `ft` or `feet`.

The precision of the elevations can be specified as well, e.g.,

In [71]:
!gridlabd geodata merge -D elevation 37.415,-122.201 --precision.elevation=2 --units=feet

id,latitude,longitude,elevation
0,37.415,-122.201,272.31


## Utility

Utility data can be obtained using the `utility` dataset.  This dataset includes information from the Homeland Infrastructure Foundation-Level Data (HIFLD).  Note that this dataset uses a very large datafile that can take some time to download the first time.

The name of the utility at a location is the default data result, e.g.,

In [86]:
!gridlabd geodata merge -D utility 37.415,-122.201

id,latitude,longitude,NAME
0,37.415,-122.201,PACIFIC GAS & ELECTRIC CO.


Other data is available (see https://hifld-geoplatform.opendata.arcgis.com/datasets/electric-retail-service-territories for detail).  A list of data available can be obtained from the dataset help, e.g., using `gridlabd geodata help utility`.  

To get the utility's summer and winter peak load and number of customers and the year for which the data is obtained, use the command

In [90]:
!gridlabd geodata merge -D utility 37.415,-122.201 --fields=WINTR_PEAK,SUMMR_PEAK,CUSTOMERS,YEAR

id,latitude,longitude,WINTR_PEAK,SUMMR_PEAK,CUSTOMERS,YEAR
0,37.415,-122.201,12624.0,17263.0,5471786.0,2018.0


The `utility` dataset also contains geometry data that can be used to generate maps, e.g.,

In [153]:
!gridlabd geodata merge -D utility 37.415,-122.201 --geometry -d #-f PLOT -o example.png -d

DEBUG [geodata/utility]: merge(args='['37.415,-122.201', '--geometry']') --> data = 
--------------------------------------------------------------------
    latitude  ...                                           geometry
id            ...                                                   
0     37.415  ...  199    MULTIPOLYGON (((-124.16609 40.80788, -1...

[1 rows x 4 columns]
--------------------------------------------------------------------
options = {
    "plot": {
        "figsize": [
            7.0,
            5.0
        ],
        "cmap": "",
        "categorical": false,
        "legend": false,
        "scheme": "",
        "k": 5,
        "vmin": NaN,
        "vmax": NaN,
        "aspect": "auto"
    },
    "show": {
        "block": true
    },
    "savefig": {
        "dpi": 300,
        "facecolor": "w",
        "edgecolor": "k",
        "orientation": "landscape",
        "format": "png",
        "transparent": false,
        "pad_inches": 0.1
    },
    "table": {

In [144]:
%%HTML 
<img src="example.png" width="50%" />

## Powerline

Powerline calculations can be performed using the `powerline` dataset.  This dataset calculates line sag, line sway, and line gallop.  To use this dataset some additional data is required. When the needed is not available at a specific location, static values can be provided using the dataset options, e.g.,

In [84]:
!gridlabd geodata merge -D powerline path_example.csv --cable_type="TACSR/AC 610mm^2" -r 100

/usr/local/lib/python3.9/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
ERROR [geodata/powerline]: cannot set using a multi-index selection indexer with a different length than the value


---
# Getting help

You can obtain help using the `help` directive, e.g.,

In [36]:
!gridlabd geodata help | head -n 10

Help on module __main__:

NAME
    __main__ - Syntax: gridlabd geodata OPTIONS DIRECTIVE [ARGUMENTS]

DESCRIPTION
    The geodata command gathers and joins geographic data. The geodata subcommand
    uses directives that are documented in the DIRECTIVES section below.
    
    In general geodata is used to acquire geographic information at a location or


In [37]:
!gridlabd geodata help distance | head -n 10

Help on module geodata_distance:

NAME
    geodata_distance - GridLAB-D Geodata Distance Package

DESCRIPTION
    The distance package computes the shortest distance between consecutive
    positions.
    
    OPTIONS


---
# Debugging

The `-d` or `--debug` option can be used to obtain details about why an error occurred.  For example, the following command has an invalid `lat,lon` tuple and results in a simple error message:

In [38]:
!gridlabd geodata merge -D distance 37,-122 38,-122 --units=furlongs

ERROR [geodata/distance]: unit 'furlongs' is not recognized


Using the debug option provides a more detailed traceback of the error that can be helpful in diagnostic the problem, particularly when it originates in a dataset package.

In [39]:
!gridlabd geodata merge -D distance 37,-122 38,-122 --units=furlongs -d

Traceback (most recent call last):
  File "/usr/local/opt/gridlabd/4.2.21-210512-develop_geodata_subcommand/bin/gridlabd-geodata", line 801, in merge
    result = MODULE.apply(data,options,config,warning)
  File "/usr/local/opt/gridlabd/4.2.21-210512-develop_geodata_subcommand/share/gridlabd/geodata_distance.py", line 103, in apply
    raise Exception(f"unit '{options['units']}' is not recognized")
Exception: unit 'furlongs' is not recognized
DEBUG [geodata/distance]: merge(args='['37,-122', '38,-122', '--units=furlongs']') --> data = 
----
None
----
options = {
    "plot": {
        "figsize": [
            7.0,
            5.0
        ],
        "cmap": "",
        "categorical": false,
        "legend": false,
        "scheme": "",
        "k": 5,
        "vmin": NaN,
        "vmax": NaN,
        "aspect": "auto"
    },
    "show": {
        "block": true
    },
    "savefig": {
        "dpi": 300,
        "facecolor": "w",
        "edgecolor": "k",
        "orientation": "landscape